<a href="https://colab.research.google.com/github/KAVYANSHTYAGI/Ransomware-Analysis-using-Machine-Learning-and-Deep-Learning/blob/main/DL_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau , LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score,classification_report
from tensorflow.keras.preprocessing.image import load_img,img_to_array

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from imblearn.over_sampling import SMOTE

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load your dataset
data_path = '/content/drive/MyDrive/ransomware_analysis_files/gan_for_synthetic/balanced_oversampled_very_noisy_extended_5k.csv'
dataset = pd.read_csv(data_path)

# Check and remove NaN values in the target variable 'Tag_y'
if dataset['Tag_y'].isnull().any():
    print("NaN values found in target variable 'Tag_y', removing rows...")
    dataset = dataset.dropna(subset=['Tag_y'])

X = dataset.drop(['Tag_y', 'Tag_x','filename', 'cryptographic_usage_encryption_algorithms','complexity_metrics_function_count', 'data_flow_collections_usage', 'hardcoded_urls', 'obfuscation_techniques_variable_name_length', 'unique_suspicious_strings'], axis=1)
y = dataset['Tag_y']


method 1

In [ ]:
# Normalize features to [0, 1]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Convert labels to categorical
y_categorical = to_categorical(y)

# Reshape data into images
num_features = X_scaled.shape[1]
next_square = int(np.ceil(np.sqrt(num_features)))  # Calculate the next perfect square
total_pixels = next_square**2
padding_required = total_pixels - num_features

# Pad the features to make the number a perfect square
X_padded = np.pad(X_scaled, ((0, 0), (0, padding_required)), 'constant')
X_images = X_padded.reshape(-1, next_square, next_square, 1)  # Reshape into (number of samples, height, width, channels)


method 2

In [3]:
import pandas as pd
import numpy as np
from PIL import Image

# Function to convert CSV to image array directly in memory
def csv_to_image_array(data, dtype=None, img_dim=(224,224), img_type='RGB'):
    images = []
    no_of_features = data.shape[1]
    sqrt_of_no_of_features = int(np.sqrt(no_of_features))
    if np.sqrt(no_of_features) > sqrt_of_no_of_features:
        sqrt_of_no_of_features += 1

    for index, row in data.iterrows():
        pixels = np.resize(row.values, (sqrt_of_no_of_features**2)).reshape(sqrt_of_no_of_features, sqrt_of_no_of_features)
        image = Image.fromarray(pixels.astype(np.uint8))
        if img_type != 'Gray':
            image = image.convert(img_type)
        image = image.resize(img_dim, Image.Resampling.NEAREST)
        images.append(np.array(image))
    return np.array(images)

# Load your dataset
data_path = '/content/drive/MyDrive/ransomware_analysis_files/gan_for_synthetic/balanced_oversampled_very_noisy_extended_5k.csv'
dataset = pd.read_csv(data_path)

# Check for NaN values in the target variable and remove them if found
if dataset['Tag_y'].isnull().any():
    print("NaN values found in target variable 'Tag_y', removing rows...")
    dataset = dataset.dropna(subset=['Tag_y'])

# Dropping unnecessary columns
features_to_drop = ['Tag_y', 'Tag_x', 'filename', 'cryptographic_usage_encryption_algorithms',
                    'complexity_metrics_function_count', 'data_flow_collections_usage', 'hardcoded_urls',
                    'obfuscation_techniques_variable_name_length', 'unique_suspicious_strings']
X_data = dataset.drop(columns=features_to_drop)
y_train = dataset['Tag_y'].values  # Assuming y_train corresponds to the rows in the CSV

# Convert the filtered CSV data into images
X_train = csv_to_image_array(X_data, dtype='uint8', img_dim=(224,224), img_type='RGB')

# Optionally, verify the shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)


<ipython-input-3-ea390ae9c461>:15: RuntimeWarning: invalid value encountered in cast
  image = Image.fromarray(pixels.astype(np.uint8))


X_train shape: (12012, 224, 224, 3)
y_train shape: (12012,)


In [4]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# Function to resize images to a new size using TensorFlow
def resize_images(images, new_size=(224, 224)):
    # Ensure images are a tensor before resizing
    images_tensor = tf.convert_to_tensor(images, dtype=tf.float32)
    return tf.image.resize(images_tensor, new_size)

# Resize X_train to (224, 224)
X_train_resized = resize_images(X_train)

# Convert y_train to categorical
y_train_categorical = to_categorical(y_train)

# Print shapes to verify everything is as expected
print("Resized X_train shape:", X_train_resized.shape)
print("Categorical y_train shape:", y_train_categorical.shape)


Resized X_train shape: (12012, 224, 224, 3)
Categorical y_train shape: (12012, 2)


In [5]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

def split_data(X, y, train_size, val_size, test_size):
    # Convert TensorFlow tensors to numpy arrays if they are not already
    if isinstance(X, tf.Tensor):
        X = X.numpy()
    if isinstance(y, tf.Tensor):
        y = y.numpy()

    # First split to separate out the training set
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=train_size, random_state=42)

    # Calculate the proportion of the remaining data to allocate to validation
    # This calculation ensures that the validation and test sets are of the correct relative size
    prop_remain = val_size / (val_size + test_size)

    # Second split to separate out the validation and test sets
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, train_size=prop_remain, random_state=42)

    # Convert arrays back to tensors if necessary
    X_train, X_val, X_test = map(tf.convert_to_tensor, [X_train, X_val, X_test])
    y_train, y_val, y_test = map(tf.convert_to_tensor, [y_train, y_val, y_test])

    return X_train, X_val, X_test, y_train, y_val, y_test

# Assuming X_train_resized and y_train_categorical are ready and correctly formatted as tensors
X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_train_resized, y_train_categorical, 0.8, 0.1, 0.1)

# Print the shapes of the datasets to verify the splits
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
print("y_test shape:", y_test.shape)


X_train shape: (9609, 224, 224, 3)
X_val shape: (1201, 224, 224, 3)
X_test shape: (1202, 224, 224, 3)
y_train shape: (9609, 2)
y_val shape: (1201, 2)
y_test shape: (1202, 2)


In [14]:
import tensorflow as tf

# Assuming y_train, y_val, and y_test are loaded as one-hot encoded arrays
# Example conversion of a one-hot encoded label tensor to sparse labels
y_train = tf.argmax(y_train, axis=1)
y_val = tf.argmax(y_val, axis=1)
y_test = tf.argmax(y_test, axis=1)


In [15]:
# Data Augmentation configuration
datagen = ImageDataGenerator(
    rotation_range=70,        # Random rotation between -10 to +10 degrees
    width_shift_range=0.1,    # Horizontal shift
    height_shift_range=0.1,   # Vertical shift
    zoom_range=0.1,           # Random zoom
    horizontal_flip=True,     # Horizontal flip
    fill_mode='nearest'  ,
    featurewise_center=True,
    featurewise_std_normalization=True
)

train_generator = datagen.flow(X_train, y_train, batch_size=64)

Custom layers

In [ ]:
# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(next_square, next_square, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(y_categorical.shape[1], activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model with data augmentation
model.fit(datagen.flow(X_train_resized, y_train, batch_size=64),
          epochs=30,
          validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

CUSTOM MODEL INSPIRED BY INCEPTIONV3

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Input
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.regularizers import l2

def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     reg_lambda=0.01):
    # Regularizer
    reg = l2(reg_lambda)

    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_regularizer=reg)(x)

    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_regularizer=reg)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_regularizer=reg)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_regularizer=reg)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_regularizer=reg)(conv_5x5)

    pool_proj = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_regularizer=reg)(pool_proj)

    output = tf.keras.layers.concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3)

    return output

input_layer = Input(shape=(224, 224, 3))

# Initial Conv and MaxPool layers
x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', kernel_regularizer=l2(0.01))(input_layer)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

# Adding Inception modules with increased filters and adjusted structure
x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=160,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=32)

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=160,
                     filters_3x3=256,
                     filters_5x5_reduce=64,
                     filters_5x5=128,
                     filters_pool_proj=64)

x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)
output_layer = Dense(2, activation='softmax')(x)  # Adjust based on your class count

model = Model(inputs=input_layer, outputs=output_layer)

# AdamW optimizer
optimizer = AdamW(learning_rate=0.001, weight_decay=0.01)

# Compile the model
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Setup callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=1e-6)
model_checkpoint = ModelCheckpoint('best_hybrid_model.keras', monitor='val_loss', save_best_only=True)

# Model summary
model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_14 (Conv2D)        │ (None, 112, 112, 64)   │          9,472 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 112, 112, 64)   │            256 │ conv2d_14[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 112, 112, 64)   │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_4           │ (None, 56, 56, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_16 (Conv2D)        │ (None, 56, 56, 96)     │          6,240 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_18 (Conv2D)        │ (None, 56, 56, 32)     │          2,080 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_5           │ (None, 56, 56, 64)     │              0 │ max_pooling2d_4[0][0]  │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_15 (Conv2D)        │ (None, 56, 56, 64)     │          4,160 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_17 (Conv2D)        │ (None, 56, 56, 160)    │        138,400 │ conv2d_16[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_19 (Conv2D)        │ (None, 56, 56, 64)     │         51,264 │ conv2d_18[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_20 (Conv2D)        │ (None, 56, 56, 32)     │          2,080 │ max_pooling2d_5[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 56, 56, 320)    │              0 │ conv2d_15[0][0],       │
│ (Concatenate)             │                        │                │ conv2d_17[0][0],       │
│                           │                        │                │ conv2d_19[0][0],       │
│                           │                        │                │ conv2d_20[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_22 (Conv2D)        │ (None, 56, 56, 160)    │         51,360 │ concatenate_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_24 (Conv2D)        │ (None, 56, 56, 64)     │         20,544 │ concatenate_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_6           │ (None, 56, 56, 320)    │              0 │ concatenate_2[0][0]    │
│ (MaxPooling2D)       

 Total params: 1,514,210 (5.78 MB)

 Trainable params: 1,514,082 (5.78 MB)

 Non-trainable params: 128 (512.00 B)

In [16]:
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr, model_checkpoint],
    shuffle=True)

# Plotting training and validation loss and accuracy
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


loss, accuracy = model.evaluate(X_val, y_val)
print(f'val Loss: {loss}, val Accuracy: {accuracy}')

loss, accuracy = model.evaluate(X_test, y_test)
print(f'test Loss: {loss}, test Accuracy: {accuracy}')

Epoch 1/100


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

Single Pre-trained models

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras import models
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

num_classes = 2

# Using MobileNetV2 as the base model with pre-trained ImageNet weights
base_model = InceptionResNetV2(include_top=False,
                         weights='imagenet',
                         input_shape=(224, 224, 3))

# Setting the base model as non-trainable
base_model.trainable = True

# Building the complete model
model = models.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.1),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='sigmoid')
])

# Configuring the optimizer
optimizer = Adam(learning_rate=1e-4)

# Compiling the model with necessary parameters
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',  # This is important for classification
              metrics=['accuracy'])

# Setting up callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=1e-6)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

# The model is now ready to be trained with model.fit(), ensuring to provide validation data and using the callbacks


219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Hybrid Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, VGG19
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

num_classes = 2

# Input shape
input_shape = (224, 224, 3)
inputs = Input(shape=input_shape)

# Load ResNet50 and VGG19 pre-trained models
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
vgg_base = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

# Set base models as trainable (or freeze as required)
resnet_base.trainable = False
vgg_base.trainable = False

# Extract features
resnet_features = GlobalAveragePooling2D()(resnet_base(inputs))
vgg_features = GlobalAveragePooling2D()(vgg_base(inputs))

# Concatenate features
combined_features = Concatenate()([resnet_features, vgg_features])

# Add a classification head
x = Dense(512, activation='relu')(combined_features)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(num_classes, activation='sigmoid')(x)

# Build the model
hybrid_model = Model(inputs, outputs)

# Configure the optimizer
optimizer = Adam(learning_rate=1e-4)

# Compile the model
hybrid_model.compile(optimizer=optimizer,
                     loss='categorical_crossentropy',  # For multi-class classification
                     metrics=['accuracy'])

# Setup callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=1e-6)
model_checkpoint = ModelCheckpoint('best_hybrid_model.keras', monitor='val_loss', save_best_only=True)

# Model summary
hybrid_model.summary()


In [ ]:
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

for X_batch, y_batch in train_generator:
    print(X_batch.shape, y_batch.shape)
    break  # Just run one iteration to check

for X_batch, y_batch in train_generator:
    print('Batch X size:', X_batch.shape, 'Batch y size:', y_batch.shape)
    if X_batch.shape[0] != y_batch.shape[0]:
        print('Mismatch found')
    break  # Stop after the first batch to check

print('Validation X size:', X_val.shape, 'Validation y size:', y_val.shape)
print('test X size:', X_test.shape, 'test y size:', y_test.shape)


Shape of X_train: (9609, 224, 224, 3)
Shape of X_test: (721, 224, 224, 3)


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:1263: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:1273: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


(64, 224, 224, 3) (64, 2)
Batch X size: (64, 224, 224, 3) Batch y size: (64, 2)
Validation X size: (1682, 224, 224, 3) Validation y size: (1682, 2)
test X size: (721, 224, 224, 3) test y size: (721, 2)


In [ ]:
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr, model_checkpoint],
    shuffle=True)

# Plotting training and validation loss and accuracy
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


loss, accuracy = model.evaluate(X_val, y_val)
print(f'val Loss: {loss}, val Accuracy: {accuracy}')

loss, accuracy = model.evaluate(X_test, y_test)
print(f'test Loss: {loss}, test Accuracy: {accuracy}')


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:1263: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:1273: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 405s 3s/step - accuracy: 0.6962 - loss: 0.5527 - val_accuracy: 0.7510 - val_loss: 0.4762 - learning_rate: 1.0000e-04
Epoch 2/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - accuracy: 0.8866 - loss: 0.2815 - val_accuracy: 0.8235 - val_loss: 0.4490 - learning_rate: 1.0000e-04
Epoch 3/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - accuracy: 0.9062 - loss: 0.2244 - val_accuracy: 0.8901 - val_loss: 0.2824 - learning_rate: 1.0000e-04
Epoch 4/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 0.9321 - loss: 0.1821 - val_accuracy: 0.9251 - val_loss: 0.1946 - learning_rate: 1.0000e-04
Epoch 5/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 0.9342 - loss: 0.1692 - val_accuracy: 0.9326 - val_loss: 0.1674 - learning_rate: 1.0000e-04
Epoch 6/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 0.9417 - loss: 0.1521 - val_accuracy: 0.9475 - val_loss: 0.1506 - learning_rate: 1.0000e-04
Epoch 7/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 

GRID SEARCH FOR HYPERPARAMETER TUNING

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, VGG19
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from itertools import product
import numpy as np

# Hyperparameter grid
learning_rates = [1e-3, 1e-4]
dense_1_units = [256, 512]
dropout_1_rates = [0.2, 0.3]
dense_2_units = [128, 256]
dropout_2_rates = [0.2, 0.3]

# Input shape
input_shape = (224, 224, 3)

# Grid Search Loop
best_model = None
best_val_accuracy = 0
best_params = {}


for lr, d1_units, d1_drop, d2_units, d2_drop in product(learning_rates, dense_1_units, dropout_1_rates, dense_2_units, dropout_2_rates):

    inputs = Input(shape=input_shape)

    resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    vgg_base = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

    resnet_base.trainable = True
    vgg_base.trainable = True

    resnet_features = GlobalAveragePooling2D()(resnet_base(inputs))
    vgg_features = GlobalAveragePooling2D()(vgg_base(inputs))

    combined_features = Concatenate()([resnet_features, vgg_features])

    x = Dense(d1_units, activation='relu')(combined_features)
    x = Dropout(d1_drop)(x)
    x = Dense(d2_units, activation='relu')(x)
    x = Dropout(d2_drop)(x)
    outputs = Dense(2, activation='sigmoid')(x)

    model = Model(inputs, outputs)
    optimizer = Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(train_generator, validation_data=(X_val, y_val), epochs=60, verbose=0)

    # Evaluate on validation data
    val_accuracy = max(history.history['val_accuracy'])
    print(f"Params: lr={lr}, d1_units={d1_units}, d1_drop={d1_drop}, d2_units={d2_units}, d2_drop={d2_drop}, val_acc={val_accuracy:.4f}")

    # Save the best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model = model
        best_params = {'learning_rate': lr, 'dense_1_units': d1_units, 'dropout_1_rate': d1_drop, 'dense_2_units': d2_units, 'dropout_2_rate': d2_drop}

print("Best Validation Accuracy:", best_val_accuracy)
print("Best Hyperparameters:", best_params)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:1263: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:1273: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


KeyboardInterrupt: 

GENETIC ALGORITHM

In [ ]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 9.5 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, VGG19
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import random
from deap import base, creator, tools, algorithms
import gc

# Enable GPU memory growth to avoid allocating all memory upfront
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Pre-trained base models are loaded once to save computation
input_shape = (224, 224, 3)
inputs = Input(shape=input_shape)

# Define global base models
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
vgg_base = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
resnet_base.trainable = False  # Freeze ResNet50
vgg_base.trainable = False  # Freeze VGG19

# Fitness Function
def evaluate(params):
    """Evaluate model accuracy for given hyperparameters."""
    # Unpack hyperparameters
    lr, d1_units, d1_drop, d2_units, d2_drop = params

    # Clear session and garbage collect to reduce memory leaks
    tf.keras.backend.clear_session()
    gc.collect()

    # Feature extraction using pre-trained models
    resnet_features = GlobalAveragePooling2D()(resnet_base(inputs))
    vgg_features = GlobalAveragePooling2D()(vgg_base(inputs))
    combined_features = Concatenate()([resnet_features, vgg_features])

    # Classification head
    x = Dense(d1_units, activation='relu')(combined_features)
    x = Dropout(d1_drop)(x)
    x = Dense(d2_units, activation='relu')(x)
    x = Dropout(d2_drop)(x)
    outputs = Dense(2, activation='sigmoid')(x)

    # Build and compile the model
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=lr),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Add EarlyStopping callback to avoid overtraining
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train the model using your train_generator and validation set
    history = model.fit(
        train_generator,
        validation_data=(X_val, y_val),
        epochs=50,  # Short epochs for quicker evaluation
        callbacks=[early_stopping],
        verbose=0  # Suppress training output for clarity
    )

    # Return the best validation accuracy
    val_accuracy = max(history.history['val_accuracy'])
    return val_accuracy,

# Genetic Algorithm Setup
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_lr", random.choice, [1e-3, 1e-4, 1e-5])
toolbox.register("attr_d1_units", random.choice, [256, 512, 1024])
toolbox.register("attr_d1_drop", random.uniform, 0.2, 0.5)
toolbox.register("attr_d2_units", random.choice, [128, 256, 512])
toolbox.register("attr_d2_drop", random.uniform, 0.2, 0.5)

toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_lr, toolbox.attr_d1_units, toolbox.attr_d1_drop,
                  toolbox.attr_d2_units, toolbox.attr_d2_drop), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Run Genetic Algorithm
population = toolbox.population(n=10)
algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=5, verbose=True)

# Get the best individual and hyperparameters
best_individual = tools.selBest(population, 1)[0]
print("Best Hyperparameters:", {
    'learning_rate': best_individual[0],
    'dense_1_units': best_individual[1],
    'dropout_1_rate': best_individual[2],
    'dense_2_units': best_individual[3],
    'dropout_2_rate': best_individual[4]
})



Physical devices cannot be modified after being initialized
